In [2]:
from sympy import *

In [9]:
dx, dy = symbols('dx, dy')
PfC, PfS, PfW, PfE, PfN = symbols('PfC, PfS, PfW, PfE, PfN')
k_ef_W, k_ef_E, k_ef_S, k_ef_N = symbols('k_ef_W, k_ef_E, k_ef_S, k_ef_N')
PtC, eta_phi = symbols('PtC, e_phi')
BCPfW, BCPfE, BCPfS, BCPfN = symbols('BCPfW, BCPfE, BCPfS, BCPfN')

qxW = (1-BCPfW) * -k_ef_W*(PfC-PfW)/dx + BCPfW * 0
qxE = (1-BCPfE) * -k_ef_E*(PfE-PfC)/dx + BCPfE * 0 
qyS = (1-BCPfS) * -k_ef_S*(PfC-PfS)/dy + BCPfS * 0
qyN = (1-BCPfN) * -k_ef_N*(PfN-PfC)/dy + BCPfN * 0
fp  = (qxE-qxW)/dx + (qyN-qyS)/dy - (PtC-PfC)/eta_phi

print(julia_code(fp.diff(PfS)))
print(julia_code(fp.diff(PfW)))
print(julia_code(fp.diff(PfC)))
print(julia_code(fp.diff(PfE)))
print(julia_code(fp.diff(PfN)))
print(julia_code(fp.diff(PtC)))

-k_ef_S .* (1 - BCPfS) ./ dy .^ 2
-k_ef_W .* (1 - BCPfW) ./ dx .^ 2
1 ./ e_phi + (k_ef_N .* (1 - BCPfN) ./ dy + k_ef_S .* (1 - BCPfS) ./ dy) ./ dy + (k_ef_E .* (1 - BCPfE) ./ dx + k_ef_W .* (1 - BCPfW) ./ dx) ./ dx
-k_ef_E .* (1 - BCPfE) ./ dx .^ 2
-k_ef_N .* (1 - BCPfN) ./ dy .^ 2
-1 ./ e_phi


In [8]:
# Total continuity
VyS, VyN, VxW, VxE = symbols('VyS, VyN, VxW, VxE')
fp = (VxE-VxW)/dx + (VyN-VyS)/dy + (PtC-PfC)/eta_phi 
print(julia_code(fp.diff(VxW)))
print(julia_code(fp.diff(VxE)))
print(julia_code(fp.diff(VyS)))
print(julia_code(fp.diff(VyN)))
print(julia_code(fp.diff(PtC)))
print(julia_code(fp.diff(PfC)))

-1 ./ dx
1 ./ dx
-1 ./ dy
1 ./ dy
1 ./ e_phi
-1 ./ e_phi


In [12]:
# Stokes x
VxC, VxW, VxE, VxS, VxN = symbols('VxC, VxW, VxE, VxS, VxN')
VySW, VySE, VyNW, VyNE  = symbols('VySW, VySE, VyNW, VyNE')
PtW, PtE = symbols('PtW, PtE')
eW, eE, eS, eN = symbols('eW, eE, eS, eN')
NeuS, DirS, NeuN, DirN = symbols('NeuS, DirS, NeuN, DirN')
#---------------------------------------#
divVW = (VxC-VxW)/dx + (VyNW-VySW)/dy
ExxW  = (VxC-VxW)/dx - Rational(1,3)*divVW
TxxW  = 2*eW*ExxW
#---------------------------------------#
divVE = (VxE-VxC)/dx + (VyNE-VySE)/dy
ExxE  = (VxE-VxC)/dx - Rational(1,3)*divVE
TxxE  = 2*eE*ExxE
#---------------------------------------#
VxS_ghost = - VxC
dVxdy = (1 - NeuS - DirS) * (VxC-VxS)/dy + NeuS * 0 + DirS * (VxC-VxS_ghost)/dy
ExyS  = Rational(1,2) * ( dVxdy + (VySE-VySW)/dx )
TxyS  = 2*eS*ExyS
#---------------------------------------#
VxN_ghost = - VxC
dVxdy = (1 - NeuN - DirN) * (VxN-VxC)/dy + NeuN * 0 + DirN * (VxN_ghost-VxC)/dy
ExyN  = Rational(1,2) * ( dVxdy + (VyNE-VyNW)/dx )
TxyN  = 2*eN*ExyN
#---------------------------------------#
f = (TxxE-TxxW)/dx + (TxyN-TxyS)/dy - (PtE-PtW)/dx
f *= -1
#---------------------------------------#
print('K[ii,iS]  = ' + julia_code(f.diff(VxS)))
print('K[ii,iW]  = ' + julia_code(f.diff(VxW)))
print('K[ii,iC]  = ' + julia_code(f.diff(VxC)))
print('K[ii,iE]  = ' + julia_code(f.diff(VxE)))
print('K[ii,iN]  = ' + julia_code(f.diff(VxN)))
print('K[ii,iSW] = ' + julia_code(f.diff(VySW)))
print('K[ii,iSE] = ' + julia_code(f.diff(VySE)))
print('K[ii,iNW] = ' + julia_code(f.diff(VyNW)))
print('K[ii,iNE] = ' + julia_code(f.diff(VyNE)))
print('K[ii,iPW] = ' + julia_code(f.diff(PtW)))
print('K[ii,iPE] = ' + julia_code(f.diff(PtE)))

K[ii,iS]  = -eS .* (-DirS - NeuS + 1) ./ dy .^ 2
K[ii,iW]  = -4 // 3 * eW ./ dx .^ 2
K[ii,iC]  = -(2 * eN .* (-DirN ./ dy - (-DirN - NeuN + 1) ./ (2 * dy)) - 2 * eS .* (DirS ./ dy + (-DirS - NeuS + 1) ./ (2 * dy))) ./ dy - (-4 // 3 * eE ./ dx - 4 // 3 * eW ./ dx) ./ dx
K[ii,iE]  = -4 // 3 * eE ./ dx .^ 2
K[ii,iN]  = -eN .* (-DirN - NeuN + 1) ./ dy .^ 2
K[ii,iSW] = -eS ./ (dx .* dy) + (2 // 3) * eW ./ (dx .* dy)
K[ii,iSE] = -2 // 3 * eE ./ (dx .* dy) + eS ./ (dx .* dy)
K[ii,iNW] = eN ./ (dx .* dy) - 2 // 3 * eW ./ (dx .* dy)
K[ii,iNE] = (2 // 3) * eE ./ (dx .* dy) - eN ./ (dx .* dy)
K[ii,iPW] = -1 ./ dx
K[ii,iPE] = 1 ./ dx


In [13]:
# Stokes y
VyC, VyW, VyE, VyS, VyN = symbols('VyC, VyW, VyE, VyS, VyN')
VxSW, VxSE, VxNW, VxNE  = symbols('VxSW, VxSE, VxNW, VxNE')
PtS, PtN = symbols('PtS, PtN')
eW, eE, eS, eN = symbols('eW, eE, eS, eN')
NeuW, DirW, NeuE, DirE = symbols('NeuW, DirW, NeuE, DirE')
#---------------------------------------#
divVS = (VyC-VyS)/dy + (VxSE-VxSW)/dx
EyyS  = (VyC-VyS)/dy - Rational(1,3)*divVS
TyyS  = 2*eS*EyyS
#---------------------------------------#
divVN = (VyN-VyC)/dy + (VxNE-VxNW)/dx
EyyN  = (VyN-VyC)/dy - Rational(1,3)*divVN
TyyN  = 2*eN*EyyN
#---------------------------------------#
VyW_ghost = -VyC
dVydx = (1 - NeuW - DirW) * (VyC-VyW)/dx + NeuW * 0 + DirW * (VyC-VyW_ghost)/dx
ExyW  = Rational(1,2) * ( dVydx + (VxNW-VxSW)/dy )
TxyW  = 2*eW*ExyW
#---------------------------------------#
VyE_ghost = -VyC
dVydx = (1 - NeuE - DirE) * (VyE-VyC)/dx + NeuE * 0 + DirE * (VyE_ghost-VyC)/dx
ExyE  = Rational(1,2) * ( dVydx + (VxNE-VxSE)/dy )
TxyE  = 2*eE*ExyE
#---------------------------------------#
f = (TyyN-TyyS)/dy + (TxyE-TxyW)/dx - (PtN-PtS)/dy
f *= -1
#---------------------------------------#
print('K[ii,iS]  = ' + julia_code(f.diff(VyS)))
print('K[ii,iW]  = ' + julia_code(f.diff(VyW)))
print('K[ii,iC]  = ' + julia_code(f.diff(VyC)))
print('K[ii,iE]  = ' + julia_code(f.diff(VyE)))
print('K[ii,iN]  = ' + julia_code(f.diff(VyN)))
print('K[ii,iSW] = ' + julia_code(f.diff(VxSW)))
print('K[ii,iSE] = ' + julia_code(f.diff(VxSE)))
print('K[ii,iNW] = ' + julia_code(f.diff(VxNW)))
print('K[ii,iNE] = ' + julia_code(f.diff(VxNE)))
print('K[ii,iPS] = ' + julia_code(f.diff(PtS)))
print('K[ii,iPN] = ' + julia_code(f.diff(PtN)))

K[ii,iS]  = -4 // 3 * eS ./ dy .^ 2
K[ii,iW]  = -eW .* (-DirW - NeuW + 1) ./ dx .^ 2
K[ii,iC]  = -(-4 // 3 * eN ./ dy - 4 // 3 * eS ./ dy) ./ dy - (2 * eE .* (-DirE ./ dx - (-DirE - NeuE + 1) ./ (2 * dx)) - 2 * eW .* (DirW ./ dx + (-DirW - NeuW + 1) ./ (2 * dx))) ./ dx
K[ii,iE]  = -eE .* (-DirE - NeuE + 1) ./ dx .^ 2
K[ii,iN]  = -4 // 3 * eN ./ dy .^ 2
K[ii,iSW] = (2 // 3) * eS ./ (dx .* dy) - eW ./ (dx .* dy)
K[ii,iSE] = eE ./ (dx .* dy) - 2 // 3 * eS ./ (dx .* dy)
K[ii,iNW] = -2 // 3 * eN ./ (dx .* dy) + eW ./ (dx .* dy)
K[ii,iNE] = -eE ./ (dx .* dy) + (2 // 3) * eN ./ (dx .* dy)
K[ii,iPS] = -1 ./ dy
K[ii,iPN] = 1 ./ dy
